In [ ]:
pip install deep-translator

In [ ]:
import os
import xml.etree.ElementTree as ET
from xml.dom import minidom
from deep_translator import GoogleTranslator

# Set paths and language
input_path = '/content/IT Safe Ways of Working_V2.xlf'  # Change as needed
lang_code = 'ko'
base_name = os.path.splitext(os.path.basename(input_path))[0]
output_path = f"/content/{base_name}_{lang_code}.xlf"

# Parse XML
tree = ET.parse(input_path)
root = tree.getroot()

# XLIFF Namespace
NS = 'urn:oasis:names:tc:xliff:document:1.2'
ET.register_namespace('', NS)
namespace = {'ns': NS}

translator = GoogleTranslator(source='auto', target=lang_code)

def translate_text(text):
    try:
        return translator.translate(text)
    except Exception as e:
        print(f"Translation error: {e}")
        return text

# Update <file> tag with target-language="ko"
file_elem = root.find('ns:file', namespace)
if file_elem is not None and 'target-language' not in file_elem.attrib:
    file_elem.attrib['target-language'] = lang_code

# For each <trans-unit>, create <target> after <source>
for trans_unit in root.findall('.//ns:trans-unit', namespace):
    source = trans_unit.find('ns:source', namespace)
    if source is None:
        continue

    # Remove existing <target> tag if present
    existing_target = trans_unit.find('ns:target', namespace)
    if existing_target is not None:
        trans_unit.remove(existing_target)

    # Create new <target> and copy/translate from <source>
    target = ET.Element('target')

    for elem in source:
        # Translate <g ctype="x-text" ...>
        if elem.tag == f'{{{NS}}}g' and elem.attrib.get('ctype') == 'x-text':
            new_elem = ET.Element(elem.tag, elem.attrib)
            if elem.text and elem.text.strip():
                new_elem.text = translate_text(elem.text.strip())
            target.append(new_elem)
        else:
            new_elem = ET.Element(elem.tag, elem.attrib)
            new_elem.text = elem.text
            target.append(new_elem)
        # Preserve tail text
        if elem.tail:
            target[-1].tail = elem.tail

    # Copy/translate text directly under <source> (if any)
    if source.text:
        target.text = source.text

    trans_unit.append(target)

# Write formatted XML output
xml_str = ET.tostring(root, encoding='utf-8')
parsed = minidom.parseString(xml_str)
pretty_xml = parsed.toprettyxml(indent=' ', encoding='utf-8')
with open(output_path, 'wb') as f:
    f.write(pretty_xml)

print(f"✅ Translation completed. Output saved to: {output_path}")


✅ Translation completed. Output saved to: /content/IT Safe Ways of Working_V2_ko.xlf
